# Computer Vision Project - Classification of Flowers


In this project your objective is to create a model in order to classify flowers. Thiszip file contains all relevant data. 

1. The data contains two folders: *train* and *test*. The *train* folder consists of 5486-images to use for training while the *test* folder contains 1351-images you can use to test your model in a **train-test-split** validation style. We have omitted another set of 1352 validation images which we will use to benchmark your final models in the last lecture. 


2. We have provided you with two label files: *train_labels.csv* and *test_labels.csv*. Each file contains the filename of the corresponding image and the class label. In total we have **102 different classes** of flowers.  You can import the label files using the `import_labels()` function provided to you in this notebook.


3. Due to the large number of images, there is a good chance that you can not easily fit the entire training and testing data into RAM. We therefore give you an implementation of a `DataGenerator` class that can be used with keras. This class will read in the images from your hard-drive for each batch during during or testing. The class comes with some nice features that could improve your training significantly such as **image resizing**, **data augmentation** and **preprocessing**. Have a look at the code to find out how.

    Initialize data generators using labels and image source directory.

    `
    datagen_train = DataGenerator('train', y_train, batch_size, input_shape, ...)
    datagen_test = DataGenerator('test', y_test, batch_size, input_shape, ...)`

    Train your model using data generators.

    `model.fit(datagen_train, validation_data=datagen_test, ...)`
    
    
4. Select a suitable model for classification. It is up to you to decide all model parameters, such as **number of layers**, **number and size of filter** in each layer, using **pooling** or, **image-size**, **data-augmentation**, **learning rate**, ... 


5. **Document** your progress and your intermediate results (your failures and improvements). Describe why you selected certain model and training parameters, what worked, what did not work. Store the training history (loss and accuracy) and create corresponding plots. This documentation will be part of your final presentation and will be **graded**.


6. Feel free to explore the internet for suitable CNN models and re-use these ideas. If you use certain features we have not touched during the lecture such as Dropout, Residual Learning or Batch Normalization. Prepare a slide in your final presentation to explain in your own (basic) terms what these things to so we can all learn from your experience. **Notice:** Very large models might perform better but will be harder and slower to train. **Do not use a pre-trained model you find online!**


7. Prepare a notebook with your model such that we can use it in the final competition. This means, store your trained model using `model.save(...)`. Your saved models can be loaded via `tf.keras.models.load_model(...)`. We will then provide you with a new folder containing images (*validation*) and a file containing labels (*validation_labels.csv*) which have the same structure. Prepare a data generator for this validation data (test it using the test data) and supply it to the 
 `evaluate_model(model, datagen)` function provided to you.
 
 Your prepared notebook could look like this:
 
    `... import stuff 
    ... code to load the stored model ...
    y_validation = import_labels('validation_labels.csv')
    datagen_validation = DataGenerator('validation', y_validation, batch_size, input_shape)
    evaluate_model(model, datagen_validation)`


8. Prepare a 15-Minute presentation of your findings and final model presentation. A rough guideline what could be interesting to your audience:
    * Explain your models architecture (number of layers, number of total parameters, how long took it to train, ...)
    * Compare the training history of your experimentats visually
    * Explain your best model (why is it better)
    * Why did you take certain decision (parameters, image size, batch size, ...)
    * What worked, what did not work (any ideas why?)
    * **What did you learn?**
    



In [2]:
# Read in label file and return a dictionary {'filename' : label}.
#
def import_labels(label_file):
    labels = dict()

    import csv
    with open(label_file) as fd:
        csvreader = csv.DictReader(fd)

        for row in csvreader:
            labels[row['filename']] = int(row['label'])
    return labels

In [3]:
import tensorflow.keras as keras
from keras.preprocessing import image
import tensorflow as tf
import os
import numpy as np


class DataGenerator(keras.utils.Sequence):

    def __init__(self, img_root_dir, labels_dict, batch_size, target_dim, preprocess_func=None, use_augmentation=False):
        self._labels_dict = labels_dict
        self._img_root_dir = img_root_dir
        self._batch_size = batch_size
        self._target_dim = target_dim
        self._preprocess_func = preprocess_func
        self._n_classes = len(set(self._labels_dict.values()))
        self._fnames_all = list(self._labels_dict.keys())
        self._use_augmentation = use_augmentation

        if self._use_augmentation:
            self._augmentor = image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest',
            )
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self._fnames_all)) / self._batch_size)

    def on_epoch_end(self):
        self._indices = np.arange(len(self._fnames_all))
        np.random.shuffle(self._indices)

    def __getitem__(self, index):
        indices = self._indices[index * self._batch_size:(index+1)*self._batch_size]

        fnames = [self._fnames_all[k] for k in indices]
        X,Y = self.__load_files__(fnames)

        return X,Y

    def __load_files__(self, batch_filenames):
        X = np.empty((self._batch_size, *self._target_dim, 3))
        Y = np.empty((self._batch_size), dtype=int)

        for idx, fname in enumerate(batch_filenames):
            img_path = os.path.join(self._img_root_dir, fname)
            img = tf.keras.utils.load_img(img_path, target_size=self._target_dim)
            x = tf.keras.utils.img_to_array(img)
           
            if self._preprocess_func is not None:
                x = self._preprocess_func(x)

            X[idx,:] = x 
            Y[idx] = self._labels_dict[fname]-1

        if self._use_augmentation:
            it = self._augmentor.flow(X, batch_size=self._batch_size, shuffle=False)
            X = it.next()

            if self._preprocess_func is not None:
                X = self._preprocess_func(X)

        return X, tf.keras.utils.to_categorical(Y, num_classes=self._n_classes)

In [4]:
def prep(arr):
    scaled_data = (arr - 127.5) / 127.5
    return scaled_data

In [5]:
data_train = DataGenerator("train",import_labels("train_labels.csv"),16,
                        (224,224),prep, use_augmentation=True)

data_test = DataGenerator("test",import_labels("test_labels.csv"),16,
                          (224,224),prep)

In [6]:
import matplotlib.pyplot as plt
def plotResults(results):
    acc = results.history['accuracy']
    val_acc = results.history['val_accuracy']
    loss = results.history['loss']
    val_loss = results.history['val_loss']
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Acc')
    plt.plot(val_acc, label='Validation Acc')
    plt.title('Training And Validation Acc')
    plt.legend(loc='lower right')
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Training And Validation Loss')
    plt.legend()
    plt.show()

In [62]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,input_shape=(224,224,3))
for i in base_model.layers[:-50]:
    i.trainable = False
model = tf.keras.models.Sequential([base_model,tf.keras.layers.GlobalAveragePooling2D(),
                                    tf.keras.layers.Dense(128,activation="relu"),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(102, activation="softmax", name="output_layer")])

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["accuracy"])
results = model.fit(data_train,epochs=50,validation_data = data_test,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10),
                                                                                tf.keras.callbacks.ModelCheckpoint("model.h5",
                                                                                                                   monitor='val_accuracy',mode='max',save_best_only=True)])

Epoch 1/50
342/342 [==============================] - 677s 2s/step - loss: 3.7999 - accuracy: 0.1873 - val_loss: 2.6056 - val_accuracy: 0.4033
Epoch 2/50
342/342 [==============================] - 667s 2s/step - loss: 2.0743 - accuracy: 0.5179 - val_loss: 1.2014 - val_accuracy: 0.7039
Epoch 3/50
342/342 [==============================] - 677s 2s/step - loss: 1.2867 - accuracy: 0.6849 - val_loss: 0.7606 - val_accuracy: 0.8095
Epoch 4/50
342/342 [==============================] - 650s 2s/step - loss: 0.8673 - accuracy: 0.7915 - val_loss: 0.4941 - val_accuracy: 0.8787
Epoch 5/50
342/342 [==============================] - 638s 2s/step - loss: 0.6398 - accuracy: 0.8480 - val_loss: 0.3098 - val_accuracy: 0.9249
Epoch 6/50
342/342 [==============================] - 660s 2s/step - loss: 0.4863 - accuracy: 0.8840 - val_loss: 0.2884 - val_accuracy: 0.9278
Epoch 7/50
342/342 [==============================] - 659s 2s/step - loss: 0.3989 - accuracy: 0.9052 - val_loss: 0.2279 - val_accuracy: 0.9375

### Flatten layer is not that important.
### LearningRateScheduler 
### EarlyStopping
### ModelCheckpoint

In [9]:
saved_model = tf.keras.models.load_model("model.h5")

_,acc= saved_model.evaluate(data_test)
acc

84/84 [==============================] - 29s 338ms/step - loss: 0.1600 - accuracy: 0.9606


0.960565447807312

In [15]:
saved_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_3   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_9 (Dense)             (None, 128)               163968    
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 output_layer (Dense)        (None, 102)               13158     
                                                                 
Total params: 2,435,110
Trainable params: 2,400,998
No

# RESNET50

In [9]:
from keras.applications.resnet import preprocess_input
data_train = DataGenerator("train",import_labels("train_labels.csv"),16,
                        (224,224),preprocess_input, use_augmentation=True)

data_test = DataGenerator("test",import_labels("test_labels.csv"),16,
                          (224,224),preprocess_input)

In [10]:
base_resnet = tf.keras.applications.ResNet50(False,"imagenet",input_shape=(224,224,3))

for i in base_resnet.layers[:-75]:
    i.trainable = False

resnet = tf.keras.models.Sequential([base_resnet,tf.keras.layers.GlobalAveragePooling2D(),
                                    tf.keras.layers.Dense(128,activation="relu"),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(102, activation="softmax", name="output_layer")])

resnet.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["accuracy"])
results = resnet.fit(data_train,epochs=50,validation_data = data_test,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10),
                                                                                tf.keras.callbacks.ModelCheckpoint("resnet_model.h5",
                                                                                                                   monitor='val_accuracy',mode='max',save_best_only=True)])

Epoch 1/50
342/342 [==============================] - 925s 3s/step - loss: 3.1780 - accuracy: 0.3162 - val_loss: 2.6738 - val_accuracy: 0.4040
Epoch 2/50
342/342 [==============================] - 969s 3s/step - loss: 1.3101 - accuracy: 0.7016 - val_loss: 2.8740 - val_accuracy: 0.4829
Epoch 3/50
342/342 [==============================] - 934s 3s/step - loss: 0.7104 - accuracy: 0.8377 - val_loss: 2.3349 - val_accuracy: 0.5350
Epoch 4/50
342/342 [==============================] - 889s 3s/step - loss: 0.4858 - accuracy: 0.8894 - val_loss: 2.6380 - val_accuracy: 0.5499
Epoch 5/50
342/342 [==============================] - 700s 2s/step - loss: 0.3812 - accuracy: 0.9101 - val_loss: 2.7205 - val_accuracy: 0.5409
Epoch 6/50
342/342 [==============================] - 849s 2s/step - loss: 0.2933 - accuracy: 0.9379 - val_loss: 2.4423 - val_accuracy: 0.5811
Epoch 7/50
342/342 [==============================] - 919s 3s/step - loss: 0.2403 - accuracy: 0.9459 - val_loss: 3.5268 - val_accuracy: 0.5461